# T3: Analize de Dados do SiaSUS

Forma: Trabalho em grupo com apresentação do ambiente e do código para o professor.

Objetivo:
Nesta primeira parte, que deve estar pronta para a aula do dia 03/11/2025, os grupos devem obter, preparar e carregar em um banco de dados relacional os dados do SIASUS (Sistema de Informações Ambulatoriais do SUS), disponibilizados publicamente pelo DataSUS.

Instruções:
1. 1) Obtenção da base de dados
 - Acesse o repositório do DataSUS e faça o download dos arquivos do SIASUS no formato .dbc.
 - Selecione os arquivos referentes a procedimentos ambulatoriais (ex.: PARS e outros complementares necessários para a descrição da tabela PARS).
2. 2) Conversão de formato
 - Converta os arquivos .dbc para .dbf utilizando as ferramentas recomendadas (ex.: TABWIN).
 - Certifique-se de validar a integridade dos arquivos após a conversão.
3) Visualização dos dados
 - Utilize o TABWIN, software oficial do DataSUS, para visualizar os arquivos .dbf.
 - A partir do TABWIN, gere:
  - Scripts SQL de criação de tabelas.
4) Carga dos dados em SGBD:
 - Insira os dados no banco de dados relacional de sua escolha (ex. MySQL, PostgreSQL, IBM DB2, etc).
 - Para arquivos menores, utilize diretamente os scripts de inserção gerados pelo TABWIN.
 - Para arquivos muito grandes (como o PARS, que contém milhões de registros), gere um arquivo CSV a partir do .dbf e utilize um comando de LOAD (carga em lote) no SGBD escolhido.
 - Se optar pelo MySQL (que foi o utilizado em aula):
  - Pesquise e utilize a instrução LOAD DATA na linha de comando do servidor (para evitar problemas de permissões com o cliente Workbench) conforme a documentação oficial: https://dev.mysql.com/doc/refman/8.4/en/load-data.html
Exemplo de comando LOAD DATA no terminal do servidor: LOAD DATA INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/PARS2508.csv' INTO TABLE pars CHARACTER SET latin1 FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES  TERMINATED BY '\r\n' IGNORE 1 ROWS; 
5) A continuidade do trabalho, na aula do dia 03/11/2025, depende do ambiente do banco de dados funcionando.

## Principais libs que usaremos: 

In [6]:
# Esse projeto foi feito com o objetivo educativo para a matéria de Programção para Ciência dos dados
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt



In [7]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# /kaggle/input/sus-data-csv/CADGERRS.csv

/kaggle/input/TB_SIGTAW.csv
/kaggle/input/PARS2501.csv
/kaggle/input/tb_municip.csv
/kaggle/input/S_CID.csv
/kaggle/input/PARS2508.csv
/kaggle/input/CBO.csv
/kaggle/input/CADGERRS.csv
/kaggle/input/rl_municip_micibge.csv
/kaggle/input/tb_micibge.csv


In [8]:
cadgerrs_data = pd.read_csv('/kaggle/input/TB_SIGTAW.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False) # Atribuição de espaços vazios.
cadgerrs_data.head(n = 5)

,IP_COD,IP_DSCR
0,101010010,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
1,101010028,ATIVIDADE EDUCATIVA / ORIENTACAO EM GRUPO NA A...
2,101010036,PRATICA CORPORAL / ATIVIDADE FISICA EM GRUPO
3,101010044,PRATICAS CORPORAIS EM MEDICINA TRADICIONAL CHI...
4,101010052,TERAPIA COMUNITARIA


In [9]:
pars2501_data = pd.read_csv('/kaggle/input/PARS2501.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False)
pars2501_data.head(n = 5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C,PA_INE,PA_NAT_JUR
0,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
1,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,R,0,NaN,0.0,0.0,0.0,121003.0,NaN,3999
2,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
3,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999
4,2254611,431720,PG,431720,0,0,0,5,0,I,...,1,K,0,NaN,0.0,0.0,0.0,NaN,NaN,3999


In [10]:
cadgerrs_data = pd.read_csv('/kaggle/input/CADGERRS.csv', na_values = 'NaN', encoding = 'latin1', low_memory = False) # Atribuição de espaços vazios.
cadgerrs_data.head(n = 5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,CNES,CPF_CNPJ,FANTASIA,RAZ_SOCI,RSOC_MAN,LOGRADOU,NUM_END,COMPLEME,BAIRRO,COD_CEP,...,EMAIL,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,CODUFMUN,EXCLUIDO,DATAINCL,DATAEXCL,NATUREZA
0,7498,65174925004,RAQUEL CRISTINA DANI,CPF 651.749.250/04-RAQUEL CRISTINA DANI,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,430920,0,20200121,99991231,NaN
1,7633,80924174072,BEATRIZ VALLE,CPF 809.241.740/72-BEATRIZ VALLE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,430920,0,20200121,99991231,NaN
2,13463,88212113061745,SAO JOAO FARMACIAS 29,CNPJ 88.212.113/0617-45-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,431690,0,20200115,99991231,NaN
3,13498,88212113076009,SAO JOAO FARMACIAS 36,CNPJ 88.212.113/0760-09-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4ª,NaN,NaN,NaN,431690,0,20200115,99991231,NaN
4,14826,88212113030351,SAO JOAO FARMACIAS 14,CNPJ 88.212.113/0303-51-COMERCIO DE MEDICAMENT...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4ª,NaN,NaN,NaN,431690,0,20200115,99991231,NaN
